In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Schizophrenia/GSE165813'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Astroblastomas exhibit radial glia stem cell lineages and differential expression of imprinted and X-inactivation escape genes [Affymetrix]"
!Series_summary	"Astroblastoma (AB) is an unusual brain tumor of unknown origin. We performed an integrated clinicogenomic analysis of 36 AB-like tumors. Lesions with MN1-BEND2 fusions demonstrated decreased promoter methylation and increased expression of IGF2-H19 and DLK1-DIO3 imprinted region genes. They also relatively overexpressed genes highly expressed during fetal brain development prior to 25 post-conception weeks (pcw), including genes enriched in ventricular zone radial glia (vRG), and generally presented in young children. Other tumors highly expressed MAP kinase pathway, PI3K pathway and X-inactivation escape genes. These and a third group of tumors tended to occur in young adults and showed enriched expression of outer radial glia (oRG) and truncated radial glia (tRG) genes, and genes highly exp

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Gene Expression Data Availability
# Based on the title and summary provided in the background information, the data involves "Affymetrix" which is a platform commonly used for gene expression data. Therefore, it is likely to contain gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Based on the sample characteristic dictionary:
#  - Looking at the keys, there is no direct or explicit mention of schizophrenia.

# Investigation of trait_row
trait_row = None  # No explicit association with Schizophrenia found

# Investigation of age_row and gender_row data unavailability
# Age and Gender information is not explicitly present in the Sample Characteristics Dictionary
age_row = None  # age data not explicitly available
gender_row = None  # gender data not explicitly available

# 2.3 Data Type Conversion
import re

def convert_trait(value):
    return None  # As trait_row is None, this function will return None

def convert_age(value):
    # since value not found, returning None stub function
    return None

def convert_gender(value):
    # since value not found, returning None stub function
    return None

# 3. Save Metadata
save_cohort_info('GSE165813', './preprocessed/Schizophrenia/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Schizophrenia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Schizophrenia/trait_data/GSE165813.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
